In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras_tuner as kt


def create_folders(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully!")
    else:
        print(f"Folder '{folder_name}' already exists!")


models_folder = 'Tuner_Models_noLHAPDF'
results_folder = 'Tuner_Results_noLHAPDF'
create_folders(models_folder)
create_folders(results_folder)
alpha = 1 / 137

data = pd.read_csv("pseudodataE288_noLHAPDF.csv")
x1_values = tf.constant(data['x1'].values, dtype=tf.float32)
x2_values = tf.constant(data['x2'].values, dtype=tf.float32)
qT_values = tf.constant(data['qT'].values, dtype=tf.float32)
QM_values = tf.constant(data['QM'].values, dtype=tf.float32)
A_true_values = tf.constant(data['A'].values, dtype=tf.float32)
f_u_x1_values = tf.constant(data['f_u_x1'].values, dtype=tf.float32)
f_ubar_x2_values = tf.constant(data['f_ubar_x2'].values, dtype=tf.float32)
f_u_x2_values = tf.constant(data['f_u_x2'].values, dtype=tf.float32)
f_ubar_x1_values = tf.constant(data['f_ubar_x1'].values, dtype=tf.float32)


def build_model(hp):
    model = models.Sequential()
    model.add(layers.Input(shape=(1,)))
    # Tune the number of layers and units
    for i in range(hp.Int("num_layers", 2, 5)):
        model.add(layers.Dense(
            units=hp.Int(f"units_{i}", min_value=32, max_value=256, step=32),
            activation=hp.Choice("activation", ["relu", "relu6", "tanh"])
        ))
    model.add(layers.Dense(1, activation="exponential"))
    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])
        ),
        loss="mae"  # Mean absolute error
    )
    return model


def custom_loss(A_true, x1, x2, qT, QM, f_u_x1, f_ubar_x2, f_u_x2, f_ubar_x1, model):
    dnnQinputs = tf.reshape(QM, (-1, 1))
    dnnQvals = model(dnnQinputs)

    pi = tf.constant(np.pi, dtype=tf.float32)
    Sk_contribution = (1 / 2) * pi * tf.exp(-qT**2 / 2)

    ux1ubarx2_term = x1 * x2 * f_u_x1 * f_ubar_x2 * Sk_contribution * dnnQvals * dnnQvals
    ubarx1ux2_term = x2 * x1 * f_u_x2 * f_ubar_x1 * Sk_contribution * dnnQvals * dnnQvals
    FUU = ux1ubarx2_term + ubarx1ux2_term
    cross_section = FUU * qT * ((4 * np.pi * alpha)**2) / (9 * QM**3)
    temploss = tf.abs(cross_section - A_true)
    loss = tf.reduce_mean(temploss)  # Mean absolute error loss
    return loss


tuner = kt.Hyperband(
    build_model,
    objective=kt.Objective("val_loss", direction="min"),
    max_epochs=50,
    factor=3,
    directory="tuning_results",
    project_name="dnnQ_hyperparameter_tuning"
)


class CustomLossWrapper(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        return custom_loss(
            A_true_values, x1_values, x2_values, qT_values, QM_values,
            f_u_x1_values, f_ubar_x2_values, f_u_x2_values, f_ubar_x1_values,
            y_pred
        )


tuner.search(x=QM_values, y=A_true_values, validation_split=0.2, verbose=1, batch_size=32)

best_hps = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.get_best_models(1)[0]

best_hps_dict = {param: best_hps.get(param) for param in best_hps.values.keys()}
best_hps_df = pd.DataFrame([best_hps_dict])
best_hps_df.to_csv(os.path.join(results_folder, "best_hyperparameters.csv"), index=False)

with open(os.path.join(results_folder, "model_architecture.json"), "w") as f:
    f.write(best_model.to_json())

print("Best hyperparameters and architecture saved!")


ModuleNotFoundError: No module named 'keras_tuner'